In [1]:
#import packages

from copy import deepcopy
import enum
from qutip.tensor import tensor
from re import T
from numpy.core.function_base import linspace
import scipy
from scipy.ndimage.measurements import label
from scipy.sparse import data
from qutip.visualization import plot_fock_distribution
from qutip.states import coherent, fock, ket2dm
import hbar_compiler
import hbar_processor
import hbar_simulation_class
import hbar_fitting
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from qutip.qip.circuit import Measurement, QubitCircuit
import qutip as qt
from qutip import basis
%matplotlib qt

define the system

In [2]:

reload(hbar_compiler)
reload(hbar_processor)
reload(hbar_simulation_class)
reload(hbar_fitting)
#qubit dimission, we only consider g and e states here
qubit_dim=2
#phonon dimission
phonon_dim=15
#how many phonon modes we consider here
phonon_num=1
#the frequency difference between qubit and phonon (qubit minus phonon)
qubit_freq=5970.04
phonon_freq=5974.115
interaction_1_freq=5972.2
interaction_3_freq=5972.96
qubit_phonon_detuning=qubit_freq-phonon_freq

#dimission of the system, qubit dimission + phonons dimission
dims=[qubit_dim]+[phonon_dim]*phonon_num
#T1 T2 we use the number for the interaction point 1
t1=[13.1]+[81]*(phonon_num)
t2=[10.1]+[134]*(phonon_num)
#pi time list for different fock state
pi_time_list=[0.9616123677058709,
 0.679329038657111,
 0.5548147810734809,
 0.48027408123596266]
#set up the processor and compiler,qb5d97 is the qubit we play around
qb_processor=hbar_processor.HBAR_processor((phonon_num+1),t1,t2,dims,g=[0.26],\
    rest_place=qubit_phonon_detuning,FSR=13)

qb_compiler = hbar_compiler.HBAR_Compiler(qb_processor.num_qubits,\
    qb_processor.params, qb_processor.pulse_dict)

qb_simulation=hbar_simulation_class.Simulation(qb_processor,qb_compiler)
qb_simulation.swap_time_list=pi_time_list

calibrate probe amplitude for qubit spec

In [ ]:
param_probe={'Omega':0.015,
    'sigma': 0.5,
    'duration':15,
    'amplitude_starkshift':0}

y_list=[]
sweep_list=np.linspace(0.005,0.04,10)
for sweep_data in sweep_list:
    param_probe['Omega']=sweep_data
    param_probe['duration']=15
    qb_simulation.ideal_phonon_fock(0)
    param_probe['amplitude_starkshift']=interaction_3_freq-qubit_freq
    qb_simulation.detuning_list=np.linspace(
        param_probe['amplitude_starkshift']-0.2,
        param_probe['amplitude_starkshift']+0.2,41)
    qb_simulation.spec_measurement(param_probe)
    y_list.append(qb_simulation.y_array)

figure,ax = plt.subplots(figsize=(8,6))
for i,sweep_data in enumerate(sweep_list):
    ax.plot(qb_simulation.detuning_list ,y_list[i],label='probe omega={}MHz'.format(sweep_data))
figure.legend()
figure.show()

simulation for different phonon drive amplitude and fit it

In [6]:
param_probe={'Omega':0.017,
    'sigma': 0.5,
    'duration':15,
    'amplitude_starkshift':0}
param_drive={'Omega':0.2,
    'sigma':0.5,
    'duration':10,
    'rotate_direction':0,
    'detuning':-qubit_phonon_detuning
    }
density_matrix_alpha_list=[]
fit_alpha_list=[]
peak_position=[]
yarray_list=[]

for i in range(10):
    peak_position.append(2.849-0.096*i)

for n,drive_amplitude in enumerate(np.linspace(0.1,0.6,6)):
    param_drive['Omega']=drive_amplitude
    qb_simulation.generate_coherent_state(param_drive)
    qb_simulation.fit_wigner()
    density_matrix_alpha_list.append(qb_simulation.alpha)
    #mixed the initial state of the qubit
    qubit_state=qb_simulation.initial_state.ptrace(0)
    phonon_state=qb_simulation.initial_state.ptrace(1)
    def keep_diag(dm):
        dim=dm.dims[0][0]
        for i in range(dim):
            projector=qt.ket2dm(fock(dim,i)) 
            if i==0:
                result=projector*phonon_state*projector
            else:
                result=result+projector*phonon_state*projector
        return result
    phonon_state=keep_diag(phonon_state)
    qb_simulation.initial_state=qt.tensor(qubit_state,phonon_state)
    param_probe['amplitude_starkshift']=interaction_3_freq-qubit_freq
    qb_simulation.detuning_list=np.linspace(
        param_probe['amplitude_starkshift']-1,
        param_probe['amplitude_starkshift']+0.2,201)
    qb_simulation.spec_measurement(param_probe)
    yarray_list.append(qb_simulation.y_array)
    result=qb_simulation.fitter.sum_lorentz_fit(peak_position,2+int(n*1.2))
    fit_alpha_list.append(result[-1])

  0%|          | 0/201 [00:00<?, ?it/s]

alpha is (0.3000000000000007-0.1999999999999993j), fidelity is 0.9969105183882101


100%|██████████| 201/201 [05:02<00:00,  1.51s/it]


len(y): 201
alpha = 0.32
alpha_err =0.002


  0%|          | 0/201 [00:00<?, ?it/s]

...

len(y): 201
alpha = 1.44
alpha_err =0.051


c:\Users\yangy\Documents\GitHub\qutip_main\qutip\visualization.py:963: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(1, 2, figsize=figsize)
  0%|          | 0/201 [00:00<?, ?it/s]

alpha is (1.8000000000000007-0.9000000000000004j), fidelity is 0.949357168652671


100%|██████████| 201/201 [07:35<00:00,  2.27s/it]


len(y): 201
alpha = 1.71
alpha_err =0.069


In [39]:
fontsize=10
density_matrix_alpha_list=np.array(density_matrix_alpha_list)
density_matrix_alpha_list=np.abs(density_matrix_alpha_list)
cm = 1/2.54
fig,ax=plt.subplots(figsize = (11*cm, 11*cm))
ax.plot(density_matrix_alpha_list,fit_alpha_list,'.')
ax.plot(density_matrix_alpha_list,linear_result[0]*density_matrix_alpha_list+linear_result[1])
ax.set_xlabel('Prepared alpha',fontsize=fontsize)
linear_result=np.polyfit(density_matrix_alpha_list,fit_alpha_list,1)
ax.set_ylabel('Poisson fitted alpha',fontsize=fontsize)

ax.set_xticks([0.5,1,1.5,2])
ax.set_xticklabels([0.5,1,1.5,2])
ax.set_yticks([0.5,1,1.5,2])
ax.set_yticklabels([0.5,1,1.5,2])
fig.gca().set_aspect('equal', adjustable='box')
fig.show()
fig.savefig('fig_calibration_phonon_drive.pdf', transparent=True)

In [ ]:
fig.show()

In [12]:
print(density_matrix_alpha_list)
print(fit_alpha_list)
linear_result

[0.36055513 0.67082039 1.02956301 1.34164079 1.65529454 2.01246118]
[0.3177733812239355, 0.593011128961417, 0.8591320466167945, 1.1488854900848666, 1.4420445900338446, 1.7098127276647852]


array([0.84947709, 0.01076195])

In [16]:
np.save('simulated_data//coherent_state_NS.npy',yarray_list)

In [3]:

param_drive={'Omega':0.2,
    'sigma':0.5,
    'duration':10,
    'rotate_direction':0,
    'detuning':-qubit_phonon_detuning
    }
density_matrix_alpha_list=[]


for n,drive_amplitude in enumerate(np.linspace(0.1,0.6,6)):
    param_drive['Omega']=drive_amplitude
    qb_simulation.generate_coherent_state(param_drive)
    qb_simulation.fit_wigner()
    density_matrix_alpha_list.append(qb_simulation.alpha)



alpha is (0.2999999999999998-0.16000000000000014j), fidelity is 0.9988902116084003
alpha is (0.6000000000000005-0.3099999999999996j), fidelity is 0.9952192531012943
alpha is (0.9000000000000004-0.45999999999999996j), fidelity is 0.9884418484663604
alpha is (1.1900000000000004-0.5999999999999996j), fidelity is 0.9787924634395501
alpha is (1.4800000000000004-0.7400000000000002j), fidelity is 0.966576217438308
alpha is (1.7599999999999998-0.8700000000000001j), fidelity is 0.952312833643998


In [4]:
density_matrix_alpha_list=np.array(density_matrix_alpha_list)
density_matrix_alpha_list=np.abs(density_matrix_alpha_list)

In [7]:
fontsize=10
cm = 1/2.54
fig,ax=plt.subplots(figsize = (11*cm, 11*cm))
fit_alpha_list=[0.3177733812239355, 0.593011128961417, 
0.8591320466167945, 1.1488854900848666, 1.4420445900338446, 1.7098127276647852]
ax.plot(density_matrix_alpha_list,fit_alpha_list,'.')
linear_result=np.polyfit(density_matrix_alpha_list,fit_alpha_list,1)
ax.plot(density_matrix_alpha_list,linear_result[0]*density_matrix_alpha_list+linear_result[1])
ax.set_xlabel('Prepared alpha',fontsize=fontsize)
ax.set_ylabel('Poisson fitted alpha',fontsize=fontsize)

ax.set_xticks([0.5,1,1.5,2])
ax.set_xticklabels([0.5,1,1.5,2])
ax.set_yticks([0.5,1,1.5,2])
ax.set_yticklabels([0.5,1,1.5,2])
fig.gca().set_aspect('equal', adjustable='box')
fig.show()
fig.savefig('fig_calibration_phonon_drive.pdf', transparent=True)


In [8]:
print(density_matrix_alpha_list)
print(fit_alpha_list)
linear_result

[0.34       0.67535176 1.0107423  1.33270402 1.6546903  1.96328806]
[0.3177733812239355, 0.593011128961417, 0.8591320466167945, 1.1488854900848666, 1.4420445900338446, 1.7098127276647852]


array([0.86089081, 0.01073611])